In [6]:
import nest
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import optimize

In [7]:
def run_network(vrest, cm, ereve, erevi, gl, tausyne, tausyni, wconde, wcondi):
    nest.set_verbosity("M_ERROR")
    spkte = np.array([50., 100., 103., 106., 109., 112., 115., 118., 121., 124., 127.]) + 150.
    spkti = spkte + 150.
    
    neuron = nest.Create('iaf_cond_exp', {'V_m': vrest, 'E_L': vrest, 'C_m': cm, 'E_ex': ereve, 'E_in': erevi, 'g_L': gl, 'tau_syn_ex': tausyne, 'tau_syn_in': tausyni, 'V_th':100.})
    spikegeneratore = nest.Create('spike_generator')
    nest.SetStatus(spikegeneratore, {'spike_times': spkte})
    spikegeneratori = nest.Create('spike_generator')
    nest.SetStatus(spikegeneratori, {'spike_times': spkti})

    voltmeter = nest.Create('voltmeter', {'interval' :0.1})
    nest.Connect(spikegeneratore, neuron, syn_spec={'weight': wconde})
    nest.Connect(spikegeneratori, neuron, syn_spec={'weight': -wcondi})
    nest.Connect(voltmeter, neuron)
    nest.print_time = True

    nest.Simulate(50.1)
    nest.SetStatus(neuron, {'I_e': 0.2})
    nest.Simulate(100.1)
    nest.SetStatus(neuron, {'I_e': 0.})
    nest.Simulate(350.1)

    voltage = nest.GetStatus(voltmeter, 'events')[0]['V_m']

    nest.ResetKernel()
    return voltage

In [9]:
data = np.load('membrane_trace_3.4_fixed.npy')[1:]

In [10]:
def plot_comparison(v):
    global data
    plt.figure()
    t = np.arange(0,len(v)*dt,dt)
    plt.plot(t, v)
    t = np.arange(0, len(data)*0.1, 0.1)
    plt.plot(t, data)

In [ ]:
#known neuron parameters
cm = 1. # nF
ereve = 20. # mV
erevi = -80. # mV

# unknown neuron paramters (random values insertet here)
vrest = -50. # mV
wconde = .0031416 # uS
wcondi = .0031416 # uS
gl = 0.042 # uS
tausyne = 3.1416 # ms
tausyni = 2*3.1416 # ms

tsim = 500.
dt = 0.1

v = run_network(vrest, cm, ereve, erevi, gl, tausyne, tausyni, wconde, wcondi)

plot_comparison(v)